In [71]:
import requests
import json
import sys
import os

In [72]:
args = sys.argv
testScriptName = args[0]
authenticate = args[1].lower()

print("Running ", args[0])

# authenticate
#if authenticate != 'y' or 'yes':
signin()

Running  /Users/temogen/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py
{}


In [73]:
def signin():
    with open(authDirectory+authfile) as authFile:
        auth = json.load(authFile)
        print(auth)
        
signin()  

{}


In [78]:
execute = 'execute'
given = 'given'
expected = 'expected'
operation = 'operation'

testDirectory = './tests/'
authDirectory = './'

authfile = 'signinauth.json'

expectedStatusCode = 200;

operations = {'GET': (lambda endpoint, request: requests.get(url=endpoint, params=request)), 
              'POST':(lambda endpoint, request: requests.post(url=endpoint, json=requset)), 
              'PUT': (lambda endpoint, request: requests.put(url=endpoint, data=request)), 
              'DELETE':(lambda endpoint, request: requests.delete(url=endpoint, request)), 
              'OPTIONS':(lambda endpoint, request: requests.options(url=endpoint))}

In [104]:
def run(testFiles):
    for testfile in testFiles:
        print("Executing ", testfile)
        with open(testDirectory + testfile) as rawTests:
            tests = json.load(rawTests)
            executeTests(tests)
            
def retrieveTestFiles():
    testFiles = [posJson for posJson in os.listdir(testDirectory) if posJson.endswith('.json')]
    return testFiles

def executeTests(tests):
    for endpoint in tests[execute]:
        for test in tests[execute][endpoint]:
            print(test, 'hitting endpoint: ', endpoint)
            request = tests[execute][endpoint][test][given]
            opt = tests[execute][endpoint][test][operation]
            response = operations[opt.upper()](endpoint, request).status_code
            
            print(test, "Completed", response, response == 200, '\n')

            
def operationSelector(operation):
    pass

In [105]:
testFiles = retrieveTestFiles()
run(testFiles)

Executing  tests1.json
test1 hitting endpoint:  https://httpbin.org/get
test1 Completed 200 True 

test2 hitting endpoint:  https://httpbin.org/get
test2 Completed 200 True 

test1 hitting endpoint:  https://httpbin.org/get2
test1 Completed 404 False 

test2 hitting endpoint:  https://httpbin.org/get2
test2 Completed 404 False 



In [101]:
''' Example Structure

{
    "execute": {
        "<url>:<port?>/<api directory>/<endpoint>": {
            "<testname>": {
                "operation": "GET",
                "given": { "item1": "thing", "item2": 1}, 
                "expected":{"item1": "returned"}
            }, 
            "<testname2>": {
                "operation": "POST",
                "given": {"item1": "thing", "item2": 1}, 
                "expected":{"item1": "returned"}
            }
        }, 
        "<url>:<port?>/<api directory>/<endpoint2>": {
             "<testname1>": {
                "operation": "PUT",
                "given": { "item1": "thing", "item2": 1}, 
                "expected":{"item1": "returned"}
            }, 
            "<testname2>": {
                "operation": "DELETE",
                "given": {"item1": "thing", "item2": 1}, 
                "expected":{"item1": "returned"}
            }
        }
    }, 
    "skip": {
        "<url>:<port?>/<api directory>/<endpoint3>": {
            "<testname>": {
                "operation": "GET",
                "given": { "item1": "thing", "item2": 1}, 
                "expected":{"item1": "returned"}
            }, 
            "<testname2>": {
                "operation": "POST",
                "given": {"item1": "thing", "item2": 1}, 
                "expected":{"item1": "returned"}
            }
        }
    }
}

'''

' Example Structure\n\n{\n    "execute": {\n        "<url>:<port?>/<api directory>/<endpoint>": {\n            "<testname>": {\n                "operation": "GET",\n                "given": { "item1": "thing", "item2": 1}, \n                "expected":{"item1": "returned"}\n            }, \n            "<testname2>": {\n                "operation": "POST",\n                "given": {"item1": "thing", "item2": 1}, \n                "expected":{"item1": "returned"}\n            }\n        }, \n        "<url>:<port?>/<api directory>/<endpoint2>": {\n             "<testname1>": {\n                "operation": "PUT",\n                "given": { "item1": "thing", "item2": 1}, \n                "expected":{"item1": "returned"}\n            }, \n            "<testname2>": {\n                "operation": "DELETE",\n                "given": {"item1": "thing", "item2": 1}, \n                "expected":{"item1": "returned"}\n            }\n        }\n    }, \n    "skip": {\n        "<url>:<port?>/